In [ ]:
import math
import random
import pandas as pd

In [ ]:
# No of players: m
m = 64

# No of queries to the oracle by the players: t
t = 20000

# Output space of the oracle: n1
# Note: n1%m = 0
n1 = t**6

# Input space of the oracle: n
n = int(((t**2)*m)/(25*math.log(m)))

#amplification factor: L
L = int(4*math.log(m,2))
# For demostartion purpose
L = 5

In [ ]:
def dfs(g,n,v,visited,count):
  for i in range(n):
    if g[v][i] != -1 and visited[i] == -1:
      visited[i] = 1
      count += 1
      visited, count = dfs(g,n,i,visited, count)
  return visited, count

def isConnected(g,n):
  visited = [-1 for i in range(n)]
  count = 0
  visited[0] = 1
  count += 1
  visited, count = dfs(g,n,0,visited,count)
  if count==n:
    return True
  else:
    return False

In [ ]:
def BFS(adj, src, dest, v, pred, dist):
  queue = []
  visited = [False for i in range(v)];

  for i in range(v):
    dist[i] = 1000000
    pred[i] = -1

  visited[src] = True;
  dist[src] = 0
  queue.append(src)

  while (len(queue) != 0):
    u = queue[0]
    queue.pop(0)
    for i in range(len(adj[u])):
      
      if (visited[adj[u][i]] == False):
        visited[adj[u][i]] = True
        dist[adj[u][i]] = dist[u] + 1
        pred[adj[u][i]] = u
        queue.append(adj[u][i])

        if (adj[u][i] == dest):
            return True

  return False;

def ShortestDistance(adj, s, dest, v):
  pred=[0 for i in range(v)]
  dist=[0 for i in range(v)]

  BFS(adj, s, dest, v, pred, dist)

  path = []
  crawl = dest
  path.append(crawl);
    
  while (pred[crawl] != -1):
    path.append(pred[crawl])
    crawl = pred[crawl]
  return path

In [ ]:
# Setup protocol
def setup(m,t,n1,n,l):
  while True:
    # Computing all key-hash pairs for all players 
    hash_table = dict()
    all_key_hash_pairs = []
    for i in range(m):
      key_hash_pairs = []
      for j in range(t):
        key = random.randrange(l*n,(l+1)*n)
        if key in hash_table:
          key_hash_pairs.append([key, hash_table[key]])
        else:
          hash_table[key] = random.randrange(l*n1, (l+1)*n1)
          key_hash_pairs.append([key, hash_table[key]])
      key_hash_pairs = sorted(key_hash_pairs, key = lambda x: x[1])
      all_key_hash_pairs.append(key_hash_pairs)
    
    # Sending hash to other players
    all_player_hash_pairs = [[] for i in range(m)]
    for i in range(m):
      for j in range(t):
        all_player_hash_pairs[all_key_hash_pairs[i][j][1]%m].append([i,all_key_hash_pairs[i][j][1]])
    all_player_hash_pairs = [sorted(player_hash_pairs, key = lambda x: x[1]) for player_hash_pairs in all_player_hash_pairs]
    
    # Detecting Collisions and storing hash values
    hash_vals = [[-1 for j in range(m)] for i in range(m)]
    for i in range(m):
      for j in range(len(all_player_hash_pairs[i])-1):
        k = j+1
        if (all_player_hash_pairs[i][j][1] == all_player_hash_pairs[i][k][1]) and (all_player_hash_pairs[i][j][0] != all_player_hash_pairs[i][k][0]):
          if hash_vals[all_player_hash_pairs[i][k][0]][all_player_hash_pairs[i][j][0]] < all_player_hash_pairs[i][j][1]: 
            hash_vals[all_player_hash_pairs[i][k][0]][all_player_hash_pairs[i][j][0]] = all_player_hash_pairs[i][j][1]
            hash_vals[all_player_hash_pairs[i][j][0]][all_player_hash_pairs[i][k][0]] = all_player_hash_pairs[i][j][1]

    # Calculating key values from hash values 
    keys = [[-1 for j in range(m)] for i in range(m)]
    for i in range(m):
      for j in range(i):
        if hash_vals[i][j] != -1:
          for key1, val in all_key_hash_pairs[i]:
            if val == hash_vals[i][j]:
              keys[i][j] = key1
              keys[j][i] = key1

    # Checking if the graph is connected or not
    if isConnected(keys,64):
      # Assigning keys to all the remaining pairs
      adj = [[] for i in range(64)]
      for i in range(64):
        for j in range(64):
          if keys[i][j] != -1:
            adj[i].append(j)
            adj[j].append(i)
      keys1 = [[-1 for j in range(m)] for i in range(m)]
      for i in range(m):
        for j in range(m):
          if i==j:
            keys1[i][j] = 0
          elif keys[i][j] != -1:
            keys1[i][j] = keys[i][j]
          else:
            path = ShortestDistance(adj, i, j, m)
            new_key = random.randrange(l*n,(l+1)*n)
            # Sending keys from source to destination
            for k in range(len(path)-1,0,-1):
              new_key = keys[k][k-1] ^ new_key
              new_key = keys[k][k-1] ^ new_key
            keys1[i][j] = new_key

      return keys1

keys = setup(m,t,n1,n,0)

In [ ]:
df_keys = pd.DataFrame(keys)
print("Keys before Amplification")
df_keys

Keys before Amplification


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,242642250,51044972,93712003,240899661,231551208,223086657,17122725,92137327,106399642,...,219497405,135577514,176280787,94518561,79031409,174511861,197653873,62423319,218349932,44618115
1,242642250,0,63247494,162625326,236923111,46062794,33464815,193827882,235858511,136003045,...,233034980,127259943,132914735,56824167,14081161,88207035,118864913,9712767,131678398,124593132
2,51044972,63247494,0,17268227,126051479,37574646,179192044,18452819,72814104,225361984,...,187307536,125954863,23937695,90210295,125632734,97201153,169565281,96419484,175151743,10184431
3,56983287,162625326,17268227,0,115729166,184341928,120418580,180825798,58149622,102080772,...,68339444,1314171,222881643,53712487,29780077,203066428,135303306,192094481,106886906,208378925
4,240899661,236923111,33963042,115729166,0,65206162,77589782,9464304,234408736,166797163,...,18510671,100566371,107844365,170510961,116007640,49548834,27985779,141938764,1681007,69757050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,174511861,88207035,97201153,203066428,49548834,132876987,240531749,125389984,201840918,17306561,...,239224512,55049425,186978586,97942846,87951646,0,70784953,207541484,51214492,31626932
60,197653873,31078596,169565281,135303306,27985779,53147821,39998098,151406867,32011120,79174665,...,60921723,152746148,126783441,103191742,90886680,70784953,0,209159024,1476690,154869250
61,177013683,9712767,96419484,15089945,141938764,144532570,77065606,34520985,117266486,120557617,...,136549336,202128236,171002607,51474158,189696909,207541484,209159024,0,159391551,58734379
62,218349932,131678398,175151743,106886906,1681007,131813534,159087911,149267743,153869043,227804200,...,53390210,149740547,232912159,182706789,58315005,81180670,182754994,159391551,0,36839507


In [ ]:
# Amplification process
def amplification(m,t,n1,n,L):
  keys = [[0 for j in range(m)] for i in range(m)]
  for l in range(L):
    keys1 = setup(m,t,n1,n,l)
    keys = [[keys[i][j] ^ keys1[i][j] for j in range(m)] for i in range(m)]
  return keys

In [ ]:
keys = amplification(m,t,n1,n,L)
df_keys = pd.DataFrame(keys)
print("Keys after Amplification")
df_keys

Keys after Amplification


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,1159380668,1485006774,1549772191,1068014287,1440314593,1013337051,1891006260,395844091,1669498790,...,69682096,1247562148,1455365963,1079909107,1305456593,1199505042,855793686,1078345037,1107516161,1951077365
1,1191317169,0,1593185522,1022939289,1217873592,1037305701,1031889933,1104005868,720525665,700763591,...,1340126711,886625849,846115259,2001691956,888451298,2101349142,928940129,274751083,54033905,1901647830
2,1485006774,667090170,0,1473654019,1558185387,681291613,899845521,1148487665,1941121519,1606051834,...,1936242015,1401006664,1072693124,1534999565,735549247,1702517084,1189339195,652860036,1270625122,1968950776
3,1542228176,875743590,1882384468,0,2002051082,1231293506,1504467114,2143046340,2014528663,1145470773,...,1588041299,961819342,572262573,1337608562,1447273118,1109331633,1365783792,1097538008,1200593112,683906186
4,1068014287,766243951,1432424029,1986705859,0,958653966,235677626,976095451,1109361967,1514329853,...,712659390,1212362178,1164009739,923121059,605469979,831732073,1197144946,1339709758,1506447402,1203292224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1291986491,1973517322,1702517084,1281824041,1040923148,1256004722,666004762,2084888174,1609469702,1333601906,...,173913391,1003921706,974793587,1160961729,692699013,0,962922710,1849503449,1082675249,1470838386
60,1016148941,961936830,1179006644,1155423484,1085706564,766048176,1115034111,968660254,535984886,1172522702,...,1934516099,789816641,1469235898,1333404748,1090952554,962922710,0,921811913,1297322758,1516650168
61,1182917584,499198765,745778731,1262774341,1339709758,1124568093,1481047311,1322075430,1340678451,1546510414,...,1353516235,1224725024,1371095830,976918216,1141298014,1849503449,921811913,0,1114848161,880192878
62,1107516161,100030566,1230478269,1200593112,1506447402,1884314093,2067544811,1097468659,1578502961,1393738078,...,1058226183,1731570638,919989703,2026261222,1194092381,1082675249,1527340111,1234058121,0,583681814
